In [7]:
"""
Probably gonna need to update this in future.
"""

objectInfoDict={'m82'     :{'redshift'    :0.000677,
                            'centerCoo'   :(148.9667,69.6794),
                            '1arcScaleBar':'17.5 pc',
                            'inclination' : 81,
                            'majorPA'     : 65,
                            'distance'    : 3.6},
                'cenA'    :{'redshift'    :0.001825,
                            'centerCoo'   :(201.3650633,-43.0191125),
                            '1arcScaleBar':'186 pc'},
                'circinus':{'redshift'    :0.001448,
                            'centerCoo'   :(213.2914583,-65.3392222),
                            '1arcScaleBar':'20.4 pc',
                            'inclination' : 65,
                            'majorPA'     : 216,
                            'distance'    : 4.2},
                'ngc253'  :{'redshift'    :0.000811,
                            'centerCoo'   :(11.888002,-25.288220),
                            '1arcScaleBar':'155 pc'},
                'ngc1068' :{'redshift'    :0.003793,
                            'centerCoo'   :(40.6696292,-0.0132806),
                            '1arcScaleBar':'528 pc'},
                'ngc3079' :{'redshift'    :0.003723,
                            'centerCoo'   :(150.4908475, 55.6797894),
                            '1arcScaleBar':'785 pc'},
                'ngc4945' :{'redshift'    :0.001878,
                            'centerCoo'   :(196.3644897,-49.4682129),
                            '1arcScaleBar':'194 pc'},
                'ngc6240' :{'redshift'    :0.024480,
                            'centerCoo'   :(253.245295,2.400926),
                            '1arcScaleBar':'4994 pc'}
               }

from dictionaryThings import saveDict
saveDict(dictionary=objectInfoDict,saveName='/Volumes/QbertPrimary/umdResearch/adapProposalNearby/objectInfoDict.pkl')

In [ ]:
"""
    This is a script which utilizes the ObsInfo class.

    Create an instance of ObsInfo using data and parameters from a single Herschel
    observation. Compute the reference velocity for the spectral axis, 
    update FITS header keywords, and save to new FITS file.
"""

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Standard
import os
import numpy as np

# Local
from ObsInfo import ObsInfo
from dictionaryThings import loadDict

In [3]:
topPath = '/Volumes/QbertPrimary/umdResearch/adapProposalNearby/'


# ----------------------------------------- #
# Necessary parameter file and dictionaries #
# ----------------------------------------- #
# Dictionary of galaxy properties
objDictName = topPath+'objectInfoDict.pkl'
# Dictionary of emission line properties.
lineDict = loadDict(topPath+'emiLineDict.pkl')
# Text file of line fitting parameters.
paramFileName = topPath + 'fittingParametersV4.txt'

In [4]:
# Which program will be reading the output FITS file?
program = '3dBarolo'
#program = 'kpvslice'

topPath = '/Volumes/QbertPrimary/umdResearch/adapProposalNearby/'


# ----------------------------------------- #
# Necessary parameter file and dictionaries #
# ----------------------------------------- #
# Dictionary of galaxy properties
objDictName = topPath+'objectInfoDict.pkl'
# Dictionary of emission line properties.
lineDict = loadDict(topPath+'emiLineDict.pkl')
# Text file of line fitting parameters.
paramFileName = topPath + 'fittingParametersV4.txt'


# -------------------------------------------------- #
# Read in the parameter file containing line profile #
# velocity limits and continuum fitting information. #
# -------------------------------------------------- #
paramFileData = np.genfromtxt(paramFileName, dtype=None, autostrip=True,
                              names=True, encoding=None)

In [5]:
for x in range(len(paramFileData)):
    # ------------------------------------ #
    # Get the galaxy and line information. #
    # ------------------------------------ #
    obsInfo = ObsInfo(x, paramFileName, objDictName)
    
    # Name of the emission line.
    lineName = paramFileData['lineNameShort'][x]
    # Rest wavelength of the emission line.
    restWave = lineDict[lineName]['restWave']
    # Spatial resolution at line wavelength
    spatRes = lineDict[lineName]['spatRes']


    # ----------------------------------------- #
    # Create the base names for saving outputs. #
    # ----------------------------------------- #
    # Base for the object's file names.
    objectNameBase = (str(obsInfo.obsId)+'_'+obsInfo.objectName
                      +'_'+lineName)
    
    # Base path to the object's folders.
    objectPathBase = (topPath+'pySpecKitCube/maybeRun3/'+obsInfo.objectName
                          +'/1arc/'+program+'/')


    # -------------------------------------------------------------- #
    # Find the minimum velocity of the spectral axis.                #
    # This will be set as the reference velocity in the FITS header. #
    # -------------------------------------------------------------- #
    fitsFile = (topPath+'pySpecKitCube/'+obsInfo.objectName+'/1arc'
                        +'/instrCorrect/fits/'+objectNameBase
                        +'_clean_correct_contFlux.fits')

    if program == 'kpvslice':
        velCorr = True
    if program == '3dBarolo':
        velCorr = False

    obsInfo.computeVels(fitsFile, restWave, velCorr=velCorr)


    # ----------------------------------------------------- #
    # Update/add spectral axis keywords in the FITS header. #
    # Also add beamsize information.                        #
    # ----------------------------------------------------- #
    fitsSavePath = objectPathBase + 'inFits/'
    if (not os.path.exists(fitsSavePath)):os.makedirs(fitsSavePath)
    
    fitsSaveName = fitsSavePath+objectNameBase+'_hdrEditVel.fits'
    
    obsInfo.updateHdr(fitsFile, restWave, beamSize=spatRes, saveFile=fitsSaveName)


    break

In [ ]:
from astropy.wcs import WCS

raCenter, decCenter = obsInfo.raCenter, obsInfo.decCenter

w = WCS(hdr).celestial
xPos, yPos = w.wcs_world2pix(raCenter, decCenter, 1)
vSys = int(waveToVel(sysWave, restWave))

print int(xPos), int(yPos), vSys